<a href="https://colab.research.google.com/github/ElwinGao4444/colab/blob/main/Tensorflow_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import keras
from keras import layers

# Sequential模型

## 模型初始化

In [3]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [   # 每个layers都可以当"函数"使用
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)

## 模型修改

In [7]:
model = keras.Sequential(name="my_sequential")  # 初始化一个空Sequential模型
model.add(layers.Dense(2, activation="relu"))   # 通过add方法，追加layer
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))
model.pop() # 通过pop方法，删除layer
print(len(model.layers))

2


## 动态weight初始化

In [8]:
layer = layers.Dense(3) # layer输出feature数为3（?行3列）
print(layer.weights)    # 定义时，不初始化weight，因为不知道输入的feature数
x = tf.ones((1, 4))     # layer输入的样本数为1，feature数为4(1行4列)
y = layer(x)            # 第一次执行时，才知道输入的feature数，才能初始化weight
print(layer.weights)    # 4行3列，代表输入feature是4，输出feature是3

[]
[<tf.Variable 'dense_8/kernel:0' shape=(4, 3) dtype=float32, numpy=
array([[ 0.85171616,  0.44975972,  0.10505223],
       [ 0.19646275, -0.36972088,  0.14572906],
       [ 0.746668  , -0.28941393, -0.16611195],
       [-0.27370858, -0.31808668, -0.6035128 ]], dtype=float32)>, <tf.Variable 'dense_8/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]


In [11]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)   # 此时无法summary，因为所有weight都未初始化
x = tf.ones((1, 4))
y = model(x)
model.summary() # 通过一次调用，所有参数初始化完成，可以看到model的完整结构

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (1, 2)                    10        
                                                                 
 dense_16 (Dense)            (1, 3)                    9         
                                                                 
 dense_17 (Dense)            (1, 4)                    16        
                                                                 
Total params: 35 (140.00 Byte)
Trainable params: 35 (140.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 静态weight初始化

In [14]:
model = keras.Sequential(
    [   # 前置一个Input对象，可以静态初始化model的结构
        keras.Input(shape=(4,)),    # Input并不是一个layer，所以不会在结构中显示
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 2)                 10        
                                                                 
 dense_25 (Dense)            (None, 3)                 9         
                                                                 
 dense_26 (Dense)            (None, 4)                 16        
                                                                 
Total params: 35 (140.00 Byte)
Trainable params: 35 (140.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model = keras.Sequential(
    [   # 也可以通过在首个layer上增加input_shape参数来指定输入feature数
        layers.Dense(2, activation="relu", input_shape=(4,)),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 2)                 10        
                                                                 
 dense_31 (Dense)            (None, 3)                 9         
                                                                 
 dense_32 (Dense)            (None, 4)                 16        
                                                                 
Total params: 35 (140.00 Byte)
Trainable params: 35 (140.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 特征提取

In [19]:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)

# 提取全部特征参数
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

# 提取指定layer的特征参数
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

## 迁移学习

In [20]:
model = keras.Sequential([
    keras.Input(shape=(784)),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10),
])
model.save_weights('my_checkpoint')

# Presumably you would want to first load pre-trained weights.
model.load_weights('my_checkpoint')

# Freeze all layers except the last one.
for layer in model.layers[:-1]:
  layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
# model.compile(...)
# model.fit(...)